# Healthy Streets of Los Angeles Injuries/Deaths by streets using LAPD DATA


Sources:
* Injuries/Deaths in the City of LA https://data.lacity.org/Public-Safety/Traffic-Collision-Data-from-2010-to-Present/d5tf-ez2w


In [3]:
# define top amount
top_n_streets = 500
top_n_x = 100

In [5]:
import pandas as pd
import datetime

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 1000)

# data

In [6]:
today = datetime.datetime.now()
print(today)

2023-12-30 16:21:55.864938


### Data preparation

In [12]:
# LAPD data is tagged with MO Codes that are booleans for specific incident peramiters, in our case these can be used to filter for specifics of a collision
crashes_raw = pd.read_csv('LAPD_crashes.csv')
crashes_raw['Severe_Injury'] = crashes_raw['MO Codes'].str.contains('3024').replace({True: 1, False: 0}).fillna(0).astype(int)
crashes_raw['Fatal_Injury'] = crashes_raw['MO Codes'].str.contains('3027').replace({True: 1, False: 0}).fillna(0).astype(int)
crashes_raw['Injury'] = crashes_raw['MO Codes'].str.contains("3026" or "3025" or "3024").replace({True: 1, False: 0}).fillna(0).astype(int)
crashes_raw['Veh vs Ped'] = crashes_raw['MO Codes'].str.contains('3003' or '3501').replace({True: 1, False: 0}).fillna(0).astype(int)
crashes_raw['Veh vs Veh'] = crashes_raw['MO Codes'].str.contains('3004').replace({True: 1, False: 0}).fillna(0).astype(int)
crashes_raw['Veh vs Bike'] = crashes_raw['MO Codes'].str.contains('3008').replace({True: 1, False: 0}).fillna(0).astype(int)
crashes_raw['Veh vs Bike'] = crashes_raw['MO Codes'].str.contains('3008').replace({True: 1, False: 0}).fillna(0).astype(int)
crashes_raw['Year'] = crashes_raw['Date Occurred'].str[-4:].astype(int)
crashes_raw['Hit and Run'] = crashes_raw['MO Codes'].str.contains('3029' or '3030').replace({True: 1, False: 0}).fillna(0).astype(int)
crashes_raw['At Intersection'] = crashes_raw['MO Codes'].str.contains('3036').replace({True: 1, False: 0}).fillna(0).astype(int)
crashes_raw['doored'] = crashes_raw['MO Codes'].str.contains('3603').replace({True: 1, False: 0}).fillna(0).astype(int)
crashes_year = crashes_raw[crashes_raw['Year'] > 2012]
display(crashes_year.head(10))

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Location,Latitude,Longitude,Year,Severe_Injury,Fatal_Injury,Injury,Veh vs Ped,Veh vs Veh,Veh vs Bike,Hit and Run,At Intersection,doored
0,190319651,08/24/2019,08/24/2019,450,3,Southwest,356,997,TRAFFIC COLLISION,3036 3004 3026 3101 4003,22.0,M,H,101.0,STREET,JEFFERSON BL,NORMANDIE AV,"(34.0255, -118.3002)",34.0255,-118.3002,2019,0,0,1,0,1,0,0,1,0
1,190319680,08/30/2019,08/30/2019,2320,3,Southwest,355,997,TRAFFIC COLLISION,3037 3006 3028 3030 3039 3101 4003,30.0,F,H,101.0,STREET,JEFFERSON BL,W WESTERN,"(34.0256, -118.3089)",34.0256,-118.3089,2019,0,0,0,0,0,0,0,0,0
2,190413769,08/25/2019,08/25/2019,545,4,Hollenbeck,422,997,TRAFFIC COLLISION,3101 3401 3701 3006 3030,NaN,M,X,101.0,STREET,N BROADWAY,W EASTLAKE AV,"(34.0738, -118.2078)",34.0738,-118.2078,2019,0,0,0,0,0,0,0,0,0
3,190127578,11/20/2019,11/20/2019,350,1,Central,128,997,TRAFFIC COLLISION,0605 3101 3401 3701 3011 3034,21.0,M,H,101.0,STREET,1ST,CENTRAL,"(34.0492, -118.2391)",34.0492,-118.2391,2019,0,0,0,0,0,0,0,0,0
4,190319695,08/30/2019,08/30/2019,2100,3,Southwest,374,997,TRAFFIC COLLISION,0605 4025 3037 3004 3025 3101,49.0,M,B,101.0,STREET,MARTIN LUTHER KING JR,ARLINGTON AV,"(34.0108, -118.3182)",34.0108,-118.3182,2019,0,0,0,0,1,0,0,0,0
5,190411883,07/06/2019,07/06/2019,950,4,Hollenbeck,423,997,TRAFFIC COLLISION,3101 3401 3701 3003 3025 3029,60.0,M,H,101.0,STREET,MAIN,JOHNSTON,"(34.066, -118.2102)",34.0660,-118.2102,2019,0,0,0,1,0,0,1,0,0
6,190514552,08/23/2019,08/23/2019,1843,5,Harbor,514,997,TRAFFIC COLLISION,0605 3037 3003 3026 3029 3101,73.0,M,H,101.0,STREET,I ST,KING AV,"(33.7807, -118.2756)",33.7807,-118.2756,2019,0,0,1,1,0,0,1,0,0
7,190319665,08/31/2019,08/30/2019,2200,3,Southwest,333,997,TRAFFIC COLLISION,4025 3037 3002 3028,39.0,M,B,101.0,STREET,CRENSHAW BL,28TH ST,"(34.0312, -118.3351)",34.0312,-118.3351,2019,0,0,0,0,0,0,0,0,0
8,190319702,08/31/2019,08/31/2019,1330,3,Southwest,356,997,TRAFFIC COLLISION,3036 4025 3004 3026 3101,48.0,M,H,101.0,STREET,WESTERN AV,36TH ST,"(34.0228, -118.3089)",34.0228,-118.3089,2019,0,0,1,0,1,0,0,1,0
9,190413937,08/29/2019,08/27/2019,1030,4,Hollenbeck,424,997,TRAFFIC COLLISION,3101 3401 3701 3006 3030,68.0,M,H,101.0,STREET,3500 VALLEY BL,NaN,"(34.07, -118.1998)",34.0700,-118.1998,2019,0,0,0,0,0,0,0,0,0


In [14]:
# filter for crashes with injuries or deaths
crashes_injured = crashes_year[(crashes_year['Severe_Injury'] > 0) | (crashes_year['Fatal_Injury'] > 0) | (crashes_year['Injury'] > 0)]

# filter for our selected streets
select_streets = ['WESTERN', 'OLYMPIC', 'VERMONT', 'VENTURA',  'SHERMAN', 'ROSCOE', 'SEPULVEDA', 'SUNSET']
# select_crashes = crashes_injured[crashes_injured['Address'].isin(select_streets) | crashes_injured['Cross Street'].isin(select_streets)].copy()
# display(select_crashes.head())

selected_streets = []

for street in select_streets:
    # filter using our selected street against address and street fields
    street_crashes = crashes_injured[crashes_injured['Address'].str.contains(street)  |  crashes_injured['Cross Street'].str.contains(street)]

    # filter just fatal pedestrian collisions 
    victim_crashes = street_crashes[(street_crashes['Fatal_Injury'] > 0) &(street_crashes['Veh vs Ped'] > 0) ]

    # filter for just incidents on streets and sidewalks (public ROW)
    Fatal_public_ROW = victim_crashes[(victim_crashes['Premise Description'] == 'SIDEWALK') | (victim_crashes['Premise Description']== 'STREET')]
    
    selected_streets.append(Fatal_public_ROW)

    Severe_Injuries = street_crashes['Severe_Injury'].sum()
    Fatal_Injury = Fatal_public_ROW['Fatal_Injury'].sum()
   
    print(street, Fatal_public_ROW.shape[0])



WESTERN 42
OLYMPIC 23
VERMONT 46
VENTURA 16
SHERMAN 22
ROSCOE 20
SEPULVEDA 29
SUNSET 28
